In [1]:
import warnings
warnings.filterwarnings("ignore")

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn import model_selection, preprocessing, linear_model

from sklearn.metrics import accuracy_score
import pandas as pd
from nltk.util import ngrams
import gensim.downloader as api

import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook

1. Создайте мешок слов с помощью
sklearn.feature_extraction.text.CountVectorizer.fit_transform(). Применим его к 'tweet_stemmed'
и 'tweet_lemmatized' отдельно.

● Игнорируем слова, частота которых в документе строго превышает порог 0.9 с
помощью max_df.

● Ограничим количество слов, попадающий в мешок, с помощью max_features =
1000.

● Исключим стоп-слова с помощью stop_words='english'.

● Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с
помощью CountVectorizer.get_feature_names().

In [2]:
combine_df = pd.read_pickle('combine_df.pkl')
combine_df

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[when, father, is, dysfunct, and, is, so, self...","[when, father, be, dysfunctional, and, be, so,..."
1,2,0.0,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, for, lyft, credit, can, not, use, caus...","[thank, for, lyft, credit, can, not, use, caus..."
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, your, majesti]","[bihday, your, majesty]"
3,4,0.0,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, ur]","[model, love, you, take, with, you, all, the, ...","[model, love, you, take, with, you, all, the, ..."
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, now, motiv]","[factsguide, society, now, motivation]"
...,...,...,...,...,...,...,...
49154,49155,NaN,thought factory left right polarisation trump ...,"[thought, factory, left, right, polarisation, ...","[thought, factory, left, right, polarisation, ...","[thought, factori, left, right, polaris, trump...","[think, factory, leave, right, polarisation, t..."
49155,49156,NaN,feeling like mermaid hairflip neverready forma...,"[feeling, like, mermaid, hairflip, neverready,...","[feeling, like, mermaid, hairflip, neverready,...","[feel, like, mermaid, hairflip, neverreadi, fo...","[feel, like, mermaid, hairflip, neverready, fo..."
49156,49157,NaN,hillary campaigned today in ohio omg used word...,"[hillary, campaigned, today, in, ohio, omg, us...","[hillary, campaigned, today, ohio, omg, used, ...","[hillari, campaign, today, in, ohio, omg, use,...","[hillary, campaign, today, in, ohio, omg, use,..."
49157,49158,NaN,happy at work conference right mindset leads t...,"[happy, at, work, conference, right, mindset, ...","[happy, work, conference, right, mindset, lead...","[happi, at, work, confer, right, mindset, lead...","[happy, at, work, conference, right, mindset, ..."


In [3]:
def listmerge(lsts):
    res=[]
    for lst in lsts:
      res.extend(lst)
    return res

Надеюсь, я правильно понял, что нужно объединить корпус

In [4]:
corpus_stemmed = listmerge(list(combine_df['tweet_stemmed']))
corpus_stemmed

['when',
 'father',
 'is',
 'dysfunct',
 'and',
 'is',
 'so',
 'selfish',
 'he',
 'drag',
 'hi',
 'kid',
 'into',
 'hi',
 'dysfunct',
 'run',
 'thank',
 'for',
 'lyft',
 'credit',
 'can',
 'not',
 'use',
 'caus',
 'they',
 'do',
 'not',
 'offer',
 'wheelchair',
 'van',
 'in',
 'pdx',
 'disapoint',
 'getthank',
 'bihday',
 'your',
 'majesti',
 'model',
 'love',
 'you',
 'take',
 'with',
 'you',
 'all',
 'the',
 'time',
 'in',
 'ur',
 'factsguid',
 'societi',
 'now',
 'motiv',
 'huge',
 'fan',
 'fare',
 'and',
 'big',
 'talk',
 'befor',
 'they',
 'leav',
 'chao',
 'and',
 'pay',
 'disput',
 'when',
 'they',
 'get',
 'there',
 'allshowandnogo',
 'camp',
 'tomorrow',
 'danni',
 'the',
 'next',
 'school',
 'year',
 'is',
 'the',
 'year',
 'for',
 'exam',
 'can',
 'not',
 'think',
 'about',
 'that',
 'school',
 'exam',
 'hate',
 'imagin',
 'actorslif',
 'revolutionschool',
 'girl',
 'we',
 'won',
 'love',
 'the',
 'land',
 'allin',
 'cav',
 'champion',
 'cleveland',
 'clevelandcavali',
 'wel

In [5]:
corpus_lemmatized = listmerge(list(combine_df['tweet_lemmatized']))
corpus_lemmatized

['when',
 'father',
 'be',
 'dysfunctional',
 'and',
 'be',
 'so',
 'selfish',
 'he',
 'drag',
 'his',
 'kid',
 'into',
 'his',
 'dysfunction',
 'run',
 'thank',
 'for',
 'lyft',
 'credit',
 'can',
 'not',
 'use',
 'cause',
 'they',
 'do',
 'not',
 'offer',
 'wheelchair',
 'vans',
 'in',
 'pdx',
 'disapointed',
 'getthanked',
 'bihday',
 'your',
 'majesty',
 'model',
 'love',
 'you',
 'take',
 'with',
 'you',
 'all',
 'the',
 'time',
 'in',
 'ur',
 'factsguide',
 'society',
 'now',
 'motivation',
 'huge',
 'fan',
 'fare',
 'and',
 'big',
 'talk',
 'before',
 'they',
 'leave',
 'chaos',
 'and',
 'pay',
 'dispute',
 'when',
 'they',
 'get',
 'there',
 'allshowandnogo',
 'camp',
 'tomorrow',
 'danny',
 'the',
 'next',
 'school',
 'year',
 'be',
 'the',
 'year',
 'for',
 'exams',
 'can',
 'not',
 'think',
 'about',
 'that',
 'school',
 'exams',
 'hate',
 'imagine',
 'actorslife',
 'revolutionschool',
 'girl',
 'we',
 'win',
 'love',
 'the',
 'land',
 'allin',
 'cavs',
 'champion',
 'clevel

In [6]:
vectorizers = dict()

In [7]:
count_vectorizer = CountVectorizer(
    ngram_range=(1, 1), 
    analyzer='word', 
    binary=False,  
    max_df=0.9,
    max_features=1000, 
    stop_words='english',
)

# Создаем the Bag-of-Words модель
bag_of_words = count_vectorizer.fit_transform(corpus_stemmed)

# Отобразим Bag-of-Words модель как DataFrame
feature_names = count_vectorizer.get_feature_names()

count_vectorizer_stemmed = pd.DataFrame(bag_of_words.toarray(), columns = feature_names)
vectorizers['count_vectorizer_stemmed'] = {'vectorizer': count_vectorizer, 'df': count_vectorizer_stemmed}
count_vectorizer_stemmed

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yeah,year,yesterday,yo,yoga,york,young,youtub,yr,yummi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586321,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
586322,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
586323,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
586324,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
count_vectorizer = CountVectorizer(
    ngram_range=(1, 1), 
    analyzer='word', 
    binary=False,  
    max_df=0.9,
    max_features=1000, 
    stop_words='english',
)

# Создаем the Bag-of-Words модель
bag_of_words = count_vectorizer.fit_transform(corpus_lemmatized)

# Отобразим Bag-of-Words модель как DataFrame
feature_names = count_vectorizer.get_feature_names()

count_vectorizer_lemmatized = pd.DataFrame(bag_of_words.toarray(), columns = feature_names)
vectorizers['count_vectorizer_lemmatized'] = {'vectorizer': count_vectorizer, 'df': count_vectorizer_lemmatized}
count_vectorizer_lemmatized

,able,absolutely,accept,account,act,action,actor,actually,adapt,add,...,yesterday,yo,yoga,york,young,youth,youtube,yr,yrs,yummy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586321,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
586322,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
586323,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
586324,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


2. Создайте мешок слов с помощью
sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). Применим его к 'tweet_stemmed' и
'tweet_lemmatized' отдельно.

● Игнорируем слова, частота которых в документе строго превышает порог 0.9 с
помощью max_df.

● Ограничим количество слов, попадающий в мешок, с помощью max_features =
1000.

● Исключим стоп-слова с помощью stop_words='english'.

● Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с
помощью TfidfVectorizer.get_feature_names()

In [9]:
tfidf_vectorizer = TfidfVectorizer(
    ngram_range=(1, 1), 
    analyzer='word', 
    binary=False,  
    max_df=0.9,
    max_features=1000, 
    stop_words='english',
)

# Создаем the Bag-of-Words модель
values = tfidf_vectorizer.fit_transform(corpus_stemmed)

# Отобразим Bag-of-Words модель как DataFrame
feature_names = tfidf_vectorizer.get_feature_names()

tfidf_vectorizer_stemmed = pd.DataFrame(values.toarray(), columns = feature_names)
vectorizers['tfidf_vectorizer_stemmed'] = {'vectorizer': tfidf_vectorizer, 'df': tfidf_vectorizer_stemmed}
tfidf_vectorizer_stemmed

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yeah,year,yesterday,yo,yoga,york,young,youtub,yr,yummi
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586321,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
586322,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
586323,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
586324,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
tfidf_vectorizer = TfidfVectorizer(
    ngram_range=(1, 1), 
    analyzer='word', 
    binary=False,  
    max_df=0.9,
    max_features=1000, 
    stop_words='english',
)

# Создаем the Bag-of-Words модель
values = tfidf_vectorizer.fit_transform(corpus_lemmatized)

# Отобразим Bag-of-Words модель как DataFrame
feature_names = tfidf_vectorizer.get_feature_names()

tfidf_vectorizer_lemmatized = pd.DataFrame(values.toarray(), columns = feature_names)
vectorizers['tfidf_vectorizer_lemmatized'] = {'vectorizer': tfidf_vectorizer, 'df': tfidf_vectorizer_lemmatized}
tfidf_vectorizer_lemmatized

,able,absolutely,accept,account,act,action,actor,actually,adapt,add,...,yesterday,yo,yoga,york,young,youth,youtube,yr,yrs,yummy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586321,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
586322,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
586323,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
586324,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
df_vectorizers = pd.DataFrame(vectorizers).T
df_vectorizers

,vectorizer,df
count_vectorizer_stemmed,"CountVectorizer(max_df=0.9, max_features=1000,...",abl absolut accept account act ac...
count_vectorizer_lemmatized,"CountVectorizer(max_df=0.9, max_features=1000,...",able absolutely accept account act...
tfidf_vectorizer_stemmed,"TfidfVectorizer(max_df=0.9, max_features=1000,...",abl absolut accept account act ac...
tfidf_vectorizer_lemmatized,"TfidfVectorizer(max_df=0.9, max_features=1000,...",able absolutely accept account act...


3. Проверьте ваши векторайзеры на корпусе который использовали на вебинаре, составьте
таблицу метод векторизации и скор который вы получили (в методах векторизации по
изменяйте параметры что бы добиться лучшего скора) обратите внимание как
падает/растёт скор при уменьшении количества фичей, и изменении параметров, так же

In [12]:
# Загружаем данные
data = open('corpus').read()
labels, texts = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split()
    labels.append(content[0])
    texts.append(" ".join(content[1:]))

# создаем df
trainDF = pd.DataFrame()
trainDF['text'] = texts
trainDF['label'] = labels
trainDF.head(5)

,text,label
0,Stuning even for the non-gamer: This sound tra...,__label__2
1,The best soundtrack ever to anything.: I'm rea...,__label__2
2,Amazing!: This soundtrack is my favorite music...,__label__2
3,Excellent Soundtrack: I truly like this soundt...,__label__2
4,"Remember, Pull Your Jaw Off The Floor After He...",__label__2


In [13]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'])
# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.transform(valid_y)

In [14]:
for name, vectorizer in zip(df_vectorizers.index, df_vectorizers['vectorizer']):
    xtrain_count =  vectorizer.transform(train_x)
    xvalid_count =  vectorizer.transform(valid_x)

    classifier = linear_model.LogisticRegression()
    classifier.fit(xtrain_count, train_y)
    predictions = classifier.predict(xvalid_count)
    acc = accuracy_score(valid_y, predictions)
    
    vectorizers[name]['accuracy'] = acc

In [15]:
df_vectorizers = pd.DataFrame(vectorizers).T
df_vectorizers

,vectorizer,df,accuracy
count_vectorizer_stemmed,"CountVectorizer(max_df=0.9, max_features=1000,...",abl absolut accept account act ac...,0.7368
count_vectorizer_lemmatized,"CountVectorizer(max_df=0.9, max_features=1000,...",able absolutely accept account act...,0.7668
tfidf_vectorizer_stemmed,"TfidfVectorizer(max_df=0.9, max_features=1000,...",abl absolut accept account act ac...,0.7428
tfidf_vectorizer_lemmatized,"TfidfVectorizer(max_df=0.9, max_features=1000,...",able absolutely accept account act...,0.7716


In [16]:
df_vectorizers['accuracy'].sort_values(ascending=False)

tfidf_vectorizer_lemmatized    0.7716
count_vectorizer_lemmatized    0.7668
tfidf_vectorizer_stemmed       0.7428
count_vectorizer_stemmed       0.7368
Name: accuracy, dtype: object